In [42]:
dataset_name = "plane-dataset" # Name of the Voxel51 Dataset to use

### Load the Voxel51 Dataset

In [32]:
#config
import fiftyone as fo
from fiftyone import ViewField as F
import os
import numpy as np
import matplotlib.pyplot as plt


dataset = fo.load_dataset(dataset_name)
dataset.reload()

### Browse the entire Voxel51 Dataset

In [24]:
dataset.reload()
session = fo.launch_app(dataset, auto=False)

Session launched. Run `session.show()` to open the App in a cell output.


### Browse a subsection of the Voxel51 Dataset

In [ ]:
#view = dataset.exists("training")

# Veiw samples where there is no label
# view = dataset.match({"model": {"$exists": False, "$eq": None}})

# View samples with a tag
view = dataset.match_tags("train")

print(view)

session.view = view


### Add Tags to Selected Samples
Use the browser to select samples and add tags to them

In [ ]:
# Create a view containing only the selected samples
selected_view = dataset.select(session.selected)
print(selected_view)
for sample in selected_view:
    sample.tags.append("training")
    sample.save() 

### Remove Tags to Selected Samples
Use the browser to select samples and remove tags from them

In [ ]:
# Create a view containing only the selected samples
selected_view = dataset.select(session.selected)
print(selected_view)
for sample in selected_view:
    sample.tags.remove("eval")
    sample.save() 

### Add a Tag to All Samples with a given Label Field

In [ ]:
view = dataset.exists("detections")
print(view)
for sample in view:
    sample.tags.append("training")
    sample.save() 

### Add a Tag to All Samples in a Dataset

In [ ]:
for sample in dataset:
    sample.tags.append("capture-5-13")
    sample.save() 

### Number of Unique Aircraft Bodies per Normalized Model

In [12]:

norm_models = dataset.distinct("norm_model.label")
for norm_model in norm_models:
    view = dataset.filter_labels("norm_model", (F("label") == norm_model)).select_fields("icao24")
    unique_aircraft = view.distinct("icao24.label")
    
    #print( unique_aircraft)
    total_predictions=0
    for icao24 in unique_aircraft:
        plane_view = dataset.filter_labels("icao24", (F("label") == icao24))

        for sample in plane_view:
            #print(sample["auto_aug_predict_tiled"])
            total_predictions = total_predictions + len(sample["auto_aug_predict_tiled"].detections)

    #print("{},{},{}".format(norm_model,len(unique_aircraft),total_predictions))     

737-400,1,46
737-700,16,769
737-800,24,1299
737-900,12,668
757-200,3,200
767-200,1,40
777-200,2,127
A300,2,108
A319,17,1132
A320,72,4051
A321,27,1506
Bombardier Challenger,10,488
CRJ200,1,44
CRJ550,2,111
CRJ700,5,207
CRJ900,9,501
Cessna Jet,17,900
EMB-135,8,295
EMB-505,6,217
ERJ-170,63,3484
Falcon,1,41
Gulfstream,8,334
Hawker 400,5,192
Hawker 800,4,212
HondaJet,1,45
King Air,1,53
LearJet,2,87
Learjet 45/60,3,118
Pilatus PC-12,4,287
Piper Cheyenne,1,71


In [64]:
import math
norm_models = dataset.distinct("norm_model.label")
for norm_model in norm_models:
    view = dataset.filter_labels("norm_model", (F("label") == norm_model)).select_fields("icao24")
    unique_aircraft = view.distinct("icao24.label")
    if len(unique_aircraft) > 1:
        print("{} Total: {} Trian: {} Test: {}".format(norm_model,len(unique_aircraft),math.floor(len(unique_aircraft)*.75),math.ceil(len(unique_aircraft)*.25)))     
   

737-700 Total: 16 Trian: 12 Test: 4
737-800 Total: 24 Trian: 18 Test: 6
737-900 Total: 12 Trian: 9 Test: 3
757-200 Total: 3 Trian: 2 Test: 1
777-200 Total: 2 Trian: 1 Test: 1
A300 Total: 2 Trian: 1 Test: 1
A319 Total: 17 Trian: 12 Test: 5
A320 Total: 72 Trian: 54 Test: 18
A321 Total: 27 Trian: 20 Test: 7
Bombardier Challenger Total: 10 Trian: 7 Test: 3
CRJ550 Total: 2 Trian: 1 Test: 1
CRJ700 Total: 5 Trian: 3 Test: 2
CRJ900 Total: 9 Trian: 6 Test: 3
Cessna Jet Total: 17 Trian: 12 Test: 5
EMB-135 Total: 8 Trian: 6 Test: 2
EMB-505 Total: 6 Trian: 4 Test: 2
ERJ-170 Total: 63 Trian: 47 Test: 16
Gulfstream Total: 8 Trian: 6 Test: 2
Hawker 400 Total: 5 Trian: 3 Test: 2
Hawker 800 Total: 4 Trian: 3 Test: 1
LearJet Total: 2 Trian: 1 Test: 1
Learjet 45/60 Total: 3 Trian: 2 Test: 1
Pilatus PC-12 Total: 4 Trian: 3 Test: 1


In [21]:
norm_models = dataset.distinct("norm_model.label")
for norm_model in norm_models:
    view = dataset.filter_labels("norm_model", (F("label") == norm_model)).select_fields("icao24")
    unique_aircraft = view.distinct("icao24.label")

    #print( unique_aircraft)
    total_predictions=0

    plane_view = dataset.filter_labels("icao24", (F("label") == unique_aircraft[0]))

    for sample in plane_view:
        #print(sample["auto_aug_predict_tiled"])
        total_predictions = total_predictions + len(sample["auto_aug_predict_tiled"].detections)

    print("{}: {}".format(norm_model,total_predictions))  

737-400: 46
737-700: 70
737-800: 36
737-900: 67
757-200: 46
767-200: 40
777-200: 58
A300: 49
A319: 44
A320: 50
A321: 62
Bombardier Challenger: 50
CRJ200: 44
CRJ550: 51
CRJ700: 68
CRJ900: 59
Cessna Jet: 66
EMB-135: 52
EMB-505: 66
ERJ-170: 108
Falcon: 41
Gulfstream: 69
Hawker 400: 44
Hawker 800: 65
HondaJet: 45
King Air: 53
LearJet: 64
Learjet 45/60: 54
Pilatus PC-12: 69
Piper Cheyenne: 71


In [19]:
norm_models = dataset.distinct("norm_model.label")
for norm_model in norm_models:
    view = dataset.filter_labels("norm_model", (F("label") == norm_model)).match(F("auto_aug_predict_tiled.detections").length()>0)
    print("{}: {}".format(norm_model,len(view)))


737-400: 46
737-700: 768
737-800: 1298
737-900: 667
757-200: 200
767-200: 40
777-200: 127
A300: 108
A319: 1132
A320: 4049
A321: 1504
Bombardier Challenger: 488
CRJ200: 44
CRJ550: 110
CRJ700: 207
CRJ900: 500
Cessna Jet: 899
EMB-135: 295
EMB-505: 217
ERJ-170: 3470
Falcon: 41
Gulfstream: 334
Hawker 400: 192
Hawker 800: 208
HondaJet: 45
King Air: 53
LearJet: 87
Learjet 45/60: 118
Pilatus PC-12: 287
Piper Cheyenne: 71


### Add Detection Evaluation

In [35]:
view = dataset.match_tags("multi_class_eval")
print(view)

Dataset:     plane-dataset
Media type:  image
Num samples: 26106
Tags:        ['capture-3-29', 'capture-3-30', 'capture-5-13', 'eval', 'model_eval', 'model_train', 'multi_class_eval', 'multi_class_train', 'train']
Sample fields:
    id:                                                                       fiftyone.core.fields.ObjectIdField
    filepath:                                                                 fiftyone.core.fields.StringField
    tags:                                                                     fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:                                                                 fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.Metadata)
    external_id:                                                              fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Classification)
    icao24:                                                                   fiftyone.core.

In [ ]:
for sample in view:
    if sample["detections"] == None:
        sample["detections"] = fo.Detections(detections=[])
        sample.save()

In [ ]:
# Evaluate `predictions` w.r.t. labels in `ground_truth` field
view = dataset.match_tags("eval")
print(view.first())
view.evaluate_detections(
    "881_40k_predict", gt_field="detections", eval_key="eval_881"
)



In [60]:
# Evaluate the detections in the `predictions` field with respect to the
# objects in the `ground_truth` field
view = dataset.match_tags("multi_class_eval")
print(view)
results = view.evaluate_detections(
    "multi_class_100_40k_predict #auto_aug_predict_tiled #multi_class_predict", gt_field="multi_class_detections", eval_key="eval_multi_new_", compute_mAP=True
)





Dataset:     plane-dataset
Media type:  image
Num samples: 15378
Tags:        ['capture-3-29', 'capture-3-30', 'capture-5-13', 'eval', 'model_eval', 'model_train', 'multi_class_eval', 'train']
Sample fields:
    id:                                                                       fiftyone.core.fields.ObjectIdField
    filepath:                                                                 fiftyone.core.fields.StringField
    tags:                                                                     fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:                                                                 fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.Metadata)
    external_id:                                                              fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Classification)
    icao24:                                                                   fiftyone.core.fields.EmbeddedDocume

In [61]:
# Get the 10 most common classes in the dataset
counts = view.count_values("multi_class_detections.detections.label")
classes = sorted(counts, key=counts.get, reverse=True)[:15]

# Print a classification report for the top-10 classes
results.print_report(classes=classes)

# Print some statistics about the total TP/FP/FN counts
print("TP: %d" % dataset.sum("eval_multi_new__tp"))
print("FP: %d" % dataset.sum("eval_multi_new__fp"))
print("FN: %d" % dataset.sum("eval_multi_new__fn"))


print("mAP: {}".format(results.mAP()))
# 0.3957

plot = results.plot_pr_curves(classes=classes)
plot.show()

plot = results.plot_confusion_matrix(classes=classes)
plot.show()

                       precision    recall  f1-score   support

              ERJ-170       0.86      0.72      0.78      2228
                 A320       0.64      0.44      0.52      2030
                 A321       0.40      0.69      0.51       806
                 A319       0.61      0.48      0.54       728
              737-800       0.54      0.33      0.41       670
           Cessna Jet       1.00      0.02      0.03       520
              737-700       0.75      0.91      0.82       420
Bombardier Challenger       0.43      0.66      0.52       270
              737-900       0.41      0.63      0.50       257
               CRJ900       0.79      0.93      0.85       202
           Hawker 400       0.00      0.00      0.00       181
           Gulfstream       0.56      0.71      0.63       168
              EMB-135       0.57      0.57      0.57       162
              777-200       0.00      0.00      0.00       127
        Learjet 45/60       0.00      0.00      0.00  

FigureWidget({
    'data': [{'hovertemplate': '<b>class: %{text}</b><br>recall: %{x}<br>precision: %{y}<extra>…

FigureWidget({
    'data': [{'mode': 'markers',
              'opacity': 0.1,
              'type': 'scatter',…

In [ ]:
session = fo.launch_app(dataset, auto=False)

# Convert to evaluation patches
eval_patches = view.to_evaluation_patches("eval")
print(eval_patches)

# View patches in the App
session.view = eval_patches

### Low confidence sample

In [ ]:
# Show samples with most false positives
# Evaluate `predictions` w.r.t. labels in `ground_truth` field
view = dataset.match_tags("eval")
print(view.first())
eval_view = (
    view
    .sort_by("auto_aug_eval_fp", reverse=True)
    .filter_labels("auto_aug_predict_tiled", F("auto_aug_eval") == "fp")
)
session.view = eval_view

### Fix Tags

In [52]:
eval_view = dataset.match_tags("multi_class_eval")
print(eval_view)
for sample in eval_view:
    try:
        sample.tags = list(filter(lambda x: x != "multi_class_eval", sample.tags))
        sample.save()
    except ValueError:
        pass

Dataset:     plane-dataset
Media type:  image
Num samples: 2519
Tags:        ['capture-3-29', 'capture-3-30', 'capture-5-13', 'eval', 'model_eval', 'model_train', 'multi_class_eval', 'train']
Sample fields:
    id:                                                                       fiftyone.core.fields.ObjectIdField
    filepath:                                                                 fiftyone.core.fields.StringField
    tags:                                                                     fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:                                                                 fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.Metadata)
    external_id:                                                              fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Classification)
    icao24:                                                                   fiftyone.core.fields.EmbeddedDocumen

In [53]:
norm_models = dataset.distinct("norm_model.label")
for norm_model in norm_models:
    view = dataset.filter_labels("norm_model", (F("label") == norm_model)).match(F("auto_aug_predict_tiled.detections").length()>0).match_tags("multi_class_train",bool=False)
    print("{}: {}".format(norm_model,len(view)))
    if len(view) >= 100:
        for sample in view:
            sample.tags.append("multi_class_eval")
            sample.save()

737-400: 46
737-700: 568
737-800: 1198
737-900: 567
757-200: 100
767-200: 40
777-200: 127
A300: 108
A319: 1032
A320: 3949
A321: 1404
Bombardier Challenger: 388
CRJ200: 44
CRJ550: 110
CRJ700: 107
CRJ900: 400
Cessna Jet: 799
EMB-135: 195
EMB-505: 117
ERJ-170: 3370
Falcon: 41
Gulfstream: 234
Hawker 400: 192
Hawker 800: 108
HondaJet: 45
King Air: 53
LearJet: 87
Learjet 45/60: 118
Pilatus PC-12: 187
Piper Cheyenne: 71


In [59]:
eval_view = dataset.match_tags("multi_class_eval")
print(eval_view)

Dataset:     plane-dataset
Media type:  image
Num samples: 15378
Tags:        ['capture-3-29', 'capture-3-30', 'capture-5-13', 'eval', 'model_eval', 'model_train', 'multi_class_eval', 'train']
Sample fields:
    id:                                                                       fiftyone.core.fields.ObjectIdField
    filepath:                                                                 fiftyone.core.fields.StringField
    tags:                                                                     fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:                                                                 fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.Metadata)
    external_id:                                                              fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Classification)
    icao24:                                                                   fiftyone.core.fields.EmbeddedDocume